<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df_temp = pd.read_csv('data/data_temp.csv', parse_dates=['Время замера'])
df_temp.head()

,key,Время замера,Температура
0,1,2019-05-03 11:16:18,1571.0
1,1,2019-05-03 11:25:53,1604.0
2,1,2019-05-03 11:29:11,1618.0
3,1,2019-05-03 11:30:01,1601.0
4,1,2019-05-03 11:30:39,1613.0


In [3]:
start_time = df_temp.sort_values('Время замера').groupby('key').first()['Время замера']
end_time = df_temp.sort_values('Время замера').groupby('key').last()['Время замера']
full_time = end_time - start_time
full_time.name = 'full_time'
full_time.head()

key
1   00:14:21
2   00:21:45
3   00:21:40
4   00:06:28
5   00:12:42
Name: full_time, dtype: timedelta64[ns]

In [4]:
df_arc = pd.read_csv('data/data_arc.csv', 
                     parse_dates=['Начало нагрева дугой', 'Конец нагрева дугой'])
df_arc.head()

,key,Начало нагрева дугой,Конец нагрева дугой,Активная мощность,Реактивная мощность
0,1,2019-05-03 11:02:14,2019-05-03 11:06:02,0.976059,0.687084
1,1,2019-05-03 11:07:28,2019-05-03 11:10:33,0.805607,0.520285
2,1,2019-05-03 11:11:44,2019-05-03 11:14:36,0.744363,0.498805
3,1,2019-05-03 11:18:14,2019-05-03 11:24:19,1.659363,1.062669
4,1,2019-05-03 11:26:09,2019-05-03 11:28:37,0.692755,0.414397


In [5]:
df_arc['time'] = (df_arc['Конец нагрева дугой'] - df_arc['Начало нагрева дугой'])
df_arc.head()

,key,Начало нагрева дугой,Конец нагрева дугой,Активная мощность,Реактивная мощность,time
0,1,2019-05-03 11:02:14,2019-05-03 11:06:02,0.976059,0.687084,00:03:48
1,1,2019-05-03 11:07:28,2019-05-03 11:10:33,0.805607,0.520285,00:03:05
2,1,2019-05-03 11:11:44,2019-05-03 11:14:36,0.744363,0.498805,00:02:52
3,1,2019-05-03 11:18:14,2019-05-03 11:24:19,1.659363,1.062669,00:06:05
4,1,2019-05-03 11:26:09,2019-05-03 11:28:37,0.692755,0.414397,00:02:28


In [6]:
arc_time = df_arc.groupby('key').agg({'time': np.sum})
arc_time.rename({'time': 'arc_time'}, axis=1, inplace=True)
arc_time.head()

,arc_time
key,
1,00:18:18
2,00:13:31
3,00:10:55
4,00:12:21
5,00:14:29


In [7]:
arc_starts = df_arc.groupby('key').count()['time']
arc_starts.name = 'arc_starts'
arc_starts.head()

key
1    5
2    4
3    5
4    4
5    4
Name: arc_starts, dtype: int64

In [8]:
df_arc['power_ratio'] = (df_arc['Активная мощность'] / df_arc['Реактивная мощность'])
df_arc['power_ratio'].head()

0    1.420582
1    1.548395
2    1.492292
3    1.561504
4    1.671717
Name: power_ratio, dtype: float64

In [9]:
df_arc['power_ratio'].describe()

count    14876.000000
mean         1.360584
std          0.173451
min         -0.000693
25%          1.257090
50%          1.376088
75%          1.480588
max          1.949831
Name: power_ratio, dtype: float64

In [10]:
mean_power_ratio = df_arc.groupby('key').mean()['power_ratio']
mean_power_ratio.head()

key
1    1.538898
2    1.551834
3    1.583120
4    1.563487
5    1.480428
Name: power_ratio, dtype: float64

In [11]:
data = pd.DataFrame(full_time).join([arc_time, arc_starts, mean_power_ratio])
data.head()

,full_time,arc_time,arc_starts,power_ratio
key,,,,
1,00:14:21,00:18:18,5.0,1.538898
2,00:21:45,00:13:31,4.0,1.551834
3,00:21:40,00:10:55,5.0,1.583120
4,00:06:28,00:12:21,4.0,1.563487
5,00:12:42,00:14:29,4.0,1.480428


In [12]:
data.describe()

,full_time,arc_time,arc_starts,power_ratio
count,3216,3214,3214.000000,3214.000000
mean,0 days 00:30:39.861629,0 days 00:13:14.579029,4.628500,1.357452
std,0 days 00:20:46.950298,0 days 00:05:32.448622,1.608647,0.132824
min,0 days 00:00:00,0 days 00:00:57,1.000000,0.676106
25%,0 days 00:19:37,0 days 00:09:31,4.000000,1.292892
50%,0 days 00:26:21,0 days 00:12:50,4.000000,1.370550
75%,0 days 00:37:15,0 days 00:16:23,6.000000,1.438292
max,0 days 06:32:17,0 days 01:09:49,16.000000,1.782164
